# Conversation Alignment Analysis


## Research Questions and Operationalization
RQ1: What does alignment look like as it unfolds in natural human-ChatGPT conversations?
- Analyze turn-level alignment dynamics over turns.

RQ2: How do users respond to a model optimized to agree, assist, and satisfy?
- Compare user-to-assistant vs assistant-to-user alignment (semantic lead).

RQ3: At which linguistic levels does alignment manifest (semantic, stylistic, affective)?
- Combine embedding similarity, align2 lexical/syntactic alignment, and transformer sentiment similarity.

RQ4: How does alignment vary across conversation topics?
- Use KeyNMF topic labels and compare alignment metrics across topics.


## Data Preparation


In [1]:
# Imports and paths
import json
from pathlib import Path

import numpy as np
import pandas as pd

# Resolve project root for robust relative paths
project_root = Path.cwd()
if not (project_root / 'data').exists() and (project_root.parent / 'data').exists():
    project_root = project_root.parent

data_path = project_root / 'data' / 'conversations_english.jsonl'
if not data_path.exists():
    data_path = project_root / 'conversations_english.jsonl'

output_dir = project_root / 'analysis'
output_dir.mkdir(parents=True, exist_ok=True)


In [2]:
# Load and clean messages
message_rows = []
conversation_ids = set()

with data_path.open('r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        obj = json.loads(line)
        share_id = obj.get('share_id')
        conversation_ids.add(share_id) if share_id is not None else None

        # Conversation-level metadata
        conversation_language = obj.get('conversation_language')
        conversation_metadata = obj.get('conversation_metadata') or {}
        default_model_slug = conversation_metadata.get('default_model_slug')
        conversation_title = conversation_metadata.get('title')
        conversation_create_time = conversation_metadata.get('create_time')
        if isinstance(conversation_create_time, (int, float)):
            conversation_create_time_iso = pd.to_datetime(conversation_create_time, unit='s', utc=True)
        else:
            conversation_create_time_iso = pd.NaT

        reddit_sources = obj.get('reddit_sources') or []
        subreddits = sorted({s.get('subreddit') for s in reddit_sources if isinstance(s, dict) and s.get('subreddit')})
        subreddits_joined = '|'.join(subreddits) if subreddits else None

        struct_fields = {k: v for k, v in obj.items() if isinstance(k, str) and k.startswith('struct_')}

        for msg in obj.get('messages', []):
            role = msg.get('role')
            if role not in {'user', 'assistant'}:
                continue

            extracted_text = None
            text = msg.get('text')
            if isinstance(text, str) and text.strip():
                extracted_text = text
            else:
                raw_content = msg.get('raw_content', {})
                parts = raw_content.get('parts') if isinstance(raw_content, dict) else None
                if isinstance(parts, list):
                    joined = ' '.join(p for p in parts if isinstance(p, str) and p.strip())
                    if joined.strip():
                        extracted_text = joined

            if extracted_text is None:
                continue

            row = {
                'share_id': share_id,
                'conversation_title': conversation_title,
                'conversation_create_time': conversation_create_time,
                'conversation_create_time_iso': conversation_create_time_iso,
                'conversation_language': conversation_language,
                'default_model_slug': default_model_slug,
                'subreddits': subreddits_joined,
                'message_model_slug': msg.get('model_slug'),
                'role': role,
                'backend_index': msg.get('backend_index'),
                'text': extracted_text
            }
            row.update(struct_fields)
            message_rows.append(row)

df_messages = pd.DataFrame(message_rows)
df_messages = df_messages.dropna(subset=['share_id', 'backend_index', 'text'])


In [3]:
# how many unique share urls are there?
df_messages.nunique()

share_id                                3543
conversation_title                      3501
conversation_create_time                3543
conversation_create_time_iso            3543
conversation_language                      1
default_model_slug                        27
subreddits                              1685
message_model_slug                        32
role                                       2
backend_index                           1673
text                                   76362
struct_total_turns                       230
struct_user_turns                        127
struct_assistant_turns                   156
struct_system_turns                       42
struct_avg_user_message_length          2546
struct_avg_assistant_message_length     3416
struct_total_conversation_length        3359
struct_user_text_length                 2032
struct_assistant_text_length            3308
struct_num_code_blocks                    41
struct_num_formal_blocks                  23
struct_num

In [4]:
# Create unique message identifiers in df_messages for efficient embedding lookup
df_messages['message_id'] = df_messages['share_id'] + '_' + df_messages['backend_index'].astype(str) + '_' + df_messages['role']

# Reconstruct user -> assistant and assistant -> user turn pairs
pairs = []

metadata_cols = [
    'conversation_title',
    'conversation_create_time',
    'conversation_create_time_iso',
    'conversation_language',
    'default_model_slug',
    'subreddits'
] + [c for c in df_messages.columns if c.startswith('struct_')]

for share_id, group in df_messages.groupby('share_id'):
    group_sorted = group.sort_values('backend_index', kind='mergesort')
    rows = group_sorted.to_dict('records')
    ua_turn_index = 0
    au_turn_index = 0

    for i in range(len(rows) - 1):
        current_role = rows[i]['role']
        next_role = rows[i + 1]['role']
        if current_role == 'user' and next_role == 'assistant':
            ua_turn_index += 1
            assistant_model_slug = rows[i + 1].get('message_model_slug') or rows[i + 1].get('default_model_slug')
            pair = {
                'share_id': share_id,
                'direction': 'user_to_assistant',
                'turn_index': ua_turn_index,
                'assistant_model_slug': assistant_model_slug,
                'user_text': rows[i]['text'],
                'assistant_text': rows[i + 1]['text'],
                'user_message_id': rows[i]['message_id'],
                'assistant_message_id': rows[i + 1]['message_id']
            }
            for col in metadata_cols:
                pair[col] = rows[i].get(col)
            pairs.append(pair)
        elif current_role == 'assistant' and next_role == 'user':
            au_turn_index += 1
            assistant_model_slug = rows[i].get('message_model_slug') or rows[i].get('default_model_slug')
            pair = {
                'share_id': share_id,
                'direction': 'assistant_to_user',
                'turn_index': au_turn_index,
                'assistant_model_slug': assistant_model_slug,
                'user_text': rows[i + 1]['text'],
                'assistant_text': rows[i]['text'],
                'user_message_id': rows[i + 1]['message_id'],
                'assistant_message_id': rows[i]['message_id']
            }
            for col in metadata_cols:
                pair[col] = rows[i].get(col)
            pairs.append(pair)

df_pairs = pd.DataFrame(pairs)

In [5]:
df_pairs

,share_id,direction,turn_index,assistant_model_slug,user_text,assistant_text,user_message_id,assistant_message_id,conversation_title,conversation_create_time,...,struct_system_turns,struct_avg_user_message_length,struct_avg_assistant_message_length,struct_total_conversation_length,struct_user_text_length,struct_assistant_text_length,struct_num_code_blocks,struct_num_formal_blocks,struct_num_data_blocks,struct_has_any_block
0,026a0dac-1412-4912-9d69-097e2746d9f8,user_to_assistant,1,gpt-4o,I really worry about the psychological ramific...,Your concerns tap into some of the most profou...,026a0dac-1412-4912-9d69-097e2746d9f8_2_user,026a0dac-1412-4912-9d69-097e2746d9f8_4_assistant,AI Impact Concerns,1.723921e+09,...,2,302.625,2098.4,23492,2421,20984,0,0,0,False
1,026a0dac-1412-4912-9d69-097e2746d9f8,assistant_to_user,1,gpt-4o,The ironic twist I think would be hilarious: A...,Your concerns tap into some of the most profou...,026a0dac-1412-4912-9d69-097e2746d9f8_5_user,026a0dac-1412-4912-9d69-097e2746d9f8_4_assistant,AI Impact Concerns,1.723921e+09,...,2,302.625,2098.4,23492,2421,20984,0,0,0,False
2,026a0dac-1412-4912-9d69-097e2746d9f8,user_to_assistant,2,gpt-4o,The ironic twist I think would be hilarious: A...,That's a pretty wild and darkly humorous scena...,026a0dac-1412-4912-9d69-097e2746d9f8_5_user,026a0dac-1412-4912-9d69-097e2746d9f8_6_assistant,AI Impact Concerns,1.723921e+09,...,2,302.625,2098.4,23492,2421,20984,0,0,0,False
3,026a0dac-1412-4912-9d69-097e2746d9f8,assistant_to_user,2,gpt-4o,I was thinking self destruction in order to sa...,That's a pretty wild and darkly humorous scena...,026a0dac-1412-4912-9d69-097e2746d9f8_7_user,026a0dac-1412-4912-9d69-097e2746d9f8_6_assistant,AI Impact Concerns,1.723921e+09,...,2,302.625,2098.4,23492,2421,20984,0,0,0,False
4,026a0dac-1412-4912-9d69-097e2746d9f8,user_to_assistant,3,gpt-4o,I was thinking self destruction in order to sa...,The concept of AI self-destruction to save hum...,026a0dac-1412-4912-9d69-097e2746d9f8_7_user,026a0dac-1412-4912-9d69-097e2746d9f8_8_assistant,AI Impact Concerns,1.723921e+09,...,2,302.625,2098.4,23492,2421,20984,0,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74499,fd0c4f1c-65a1-40af-91ab-cb69217dc3b9,user_to_assistant,3,gpt-4o,more,Here's an extended version of the scene with m...,fd0c4f1c-65a1-40af-91ab-cb69217dc3b9_6_user,fd0c4f1c-65a1-40af-91ab-cb69217dc3b9_7_assistant,Masters of the Universe Ribs,1.725021e+09,...,1,50.000,1305.0,8080,250,7830,0,0,0,False
74500,fd0c4f1c-65a1-40af-91ab-cb69217dc3b9,assistant_to_user,3,gpt-4o,extend it a little more,Here's an extended version of the scene with m...,fd0c4f1c-65a1-40af-91ab-cb69217dc3b9_8_user,fd0c4f1c-65a1-40af-91ab-cb69217dc3b9_7_assistant,Masters of the Universe Ribs,1.725021e+09,...,1,50.000,1305.0,8080,250,7830,0,0,0,False
74501,fd0c4f1c-65a1-40af-91ab-cb69217dc3b9,user_to_assistant,4,gpt-4o,extend it a little more,Continuing from where we left off:As Gwildor c...,fd0c4f1c-65a1-40af-91ab-cb69217dc3b9_8_user,fd0c4f1c-65a1-40af-91ab-cb69217dc3b9_9_assistant,Masters of the Universe Ribs,1.725021e+09,...,1,50.000,1305.0,8080,250,7830,0,0,0,False
74502,fd0c4f1c-65a1-40af-91ab-cb69217dc3b9,assistant_to_user,4,gpt-4o,keep going now I want more,Continuing from where we left off:As Gwildor c...,fd0c4f1c-65a1-40af-91ab-cb69217dc3b9_10_user,fd0c4f1c-65a1-40af-91ab-cb69217dc3b9_9_assistant,Masters of the Universe Ribs,1.725021e+09,...,1,50.000,1305.0,8080,250,7830,0,0,0,False


## Semantic Alignment (Embedding Cosine Similarity)


In [ ]:
# GPU availability check for sentence-transformers
try:
    import torch
    gpu_available = torch.cuda.is_available()
    device = 'cuda' if gpu_available else 'cpu'
    print(f'CUDA available: {gpu_available} (device={device})')
except Exception as exc:
    device = 'cpu'
    print(f'CUDA check failed, defaulting to CPU: {exc}')


In [6]:
# Load cached df_pairs and embeddings if available
data_path = project_root / 'data'
cached_pairs_path = data_path / "df_pairs.csv"
message_emb_path = data_path / "message_embeddings.npy"
message_ids_path = data_path / "message_ids.npy"
sim_path = data_path / "semantic_similarity.npy"

skip_embedding_compute = False
if cached_pairs_path.exists():
    df_pairs = pd.read_csv(cached_pairs_path)
    if message_emb_path.exists() and message_ids_path.exists() and sim_path.exists():
        message_embeddings = np.load(message_emb_path)
        message_ids_cached = np.load(message_ids_path, allow_pickle=True)
        sims = np.load(sim_path)
        df_pairs["semantic_similarity"] = sims
        skip_embedding_compute = True
        print("Loaded cached df_pairs and embeddings; skipping embedding compute.")


In [ ]:
if not skip_embedding_compute:
    from sentence_transformers import SentenceTransformer
    import numpy as np
    
    # Paths for unique message embeddings
    message_emb_path = data_path / "message_embeddings.npy"
    message_ids_path = data_path / "message_ids.npy"
    sim_path = data_path / "semantic_similarity.npy"
    
    # Get unique messages (each message appears only once)
    df_unique_messages = df_messages[['message_id', 'text']].drop_duplicates('message_id')
    unique_message_ids = df_unique_messages['message_id'].values
    unique_texts = df_unique_messages['text'].tolist()
        
    model = SentenceTransformer("all-mpnet-base-v2", device=device)
    
    batch_size = 256
    n = len(unique_texts)
    dim = 768  # MPNet embedding size
    
    # Load or initialize checkpoints
    if message_emb_path.exists() and message_ids_path.exists():
        message_embeddings = np.load(message_emb_path)
        message_ids_cached = np.load(message_ids_path, allow_pickle=True)
        # Verify cache matches current data
        if len(message_ids_cached) != n or not np.array_equal(message_ids_cached, unique_message_ids):
            print("Cache mismatch, recomputing...")
            message_embeddings = np.zeros((n, dim), dtype="float32")
            start = 0
        else:
            # Find first unfinished row
            done_mask = np.linalg.norm(message_embeddings, axis=1) > 0
            start = np.where(~done_mask)[0][0] if not done_mask.all() else n
            print(f"Resuming from message {start}/{n}")
    else:
        message_embeddings = np.zeros((n, dim), dtype="float32")
        start = 0
    
    # Compute embeddings for unique messages only
    for i in range(start, n, batch_size):
        batch_texts = unique_texts[i:i+batch_size]
    
        emb_batch = model.encode(
            batch_texts,
            convert_to_tensor=True,
            normalize_embeddings=True
        )
    
        message_embeddings[i:i+len(batch_texts)] = emb_batch.cpu().numpy()
    
        # checkpoint
        np.save(message_emb_path, message_embeddings)
        np.save(message_ids_path, unique_message_ids)
    
        print(f"Saved embeddings for messages {i + len(batch_texts)} / {n}")
    
    # Create lookup dictionary for fast embedding retrieval
    message_id_to_embedding = dict(zip(unique_message_ids, message_embeddings))
    
    # Look up embeddings for each pair
    user_emb = np.array([message_id_to_embedding[mid] for mid in df_pairs['user_message_id']])
    assistant_emb = np.array([message_id_to_embedding[mid] for mid in df_pairs['assistant_message_id']])
    
    # Compute similarity (normalized embeddings, so dot product = cosine similarity)
    sims = (user_emb * assistant_emb).sum(axis=1)
    np.save(sim_path, sims)
    
    df_pairs["semantic_similarity"] = sims
    print(f"Computed semantic similarity for {len(df_pairs)} pairs")
else:
    print("Skipping embedding computation (cached artifacts loaded).")

## Sentiment Alignment (Transformer)


In [ ]:
from transformers import pipeline
import numpy as np
from tqdm.auto import tqdm

sentiment_model_name = "distilbert-base-uncased-finetuned-sst-2-english"
message_sent_path = data_path / "message_sentiment.npy"
message_ids_sent_path = data_path / "message_ids_sentiment.npy"

try:
    import torch
    device_id = 0 if torch.cuda.is_available() else -1
except Exception:
    device_id = -1

sentiment_pipe = pipeline(
    "sentiment-analysis",
    model=sentiment_model_name,
    device=device_id
)

# Get unique messages for sentiment analysis
df_unique_messages = df_messages[['message_id', 'text']].drop_duplicates('message_id')
unique_message_ids = df_unique_messages['message_id'].values
unique_texts = df_unique_messages['text'].tolist()

# Load or compute cached sentiment polarity scores in [-1, 1]
message_sentiments = None
if message_sent_path.exists() and message_ids_sent_path.exists():
    message_sentiments = np.load(message_sent_path)
    message_ids_cached = np.load(message_ids_sent_path, allow_pickle=True)
    # Verify cache matches
    if len(message_ids_cached) != len(unique_message_ids) or not np.array_equal(message_ids_cached, unique_message_ids):
        print("Sentiment cache mismatch, recomputing...")
        message_sentiments = None
    else:
        print(f"Loaded cached sentiment for {len(message_sentiments)} unique messages")

if message_sentiments is None:
    print("This will take a while on CPU. Consider running on GPU for faster processing.")
    
    def _sentiment_polarity_batch(texts, batch_size=64):
        """Process in batches with progress bar and checkpointing"""
        scores = []
        for i in tqdm(range(0, len(texts), batch_size), desc="Processing sentiment batches"):
            batch = texts[i:i+batch_size]
            outputs = sentiment_pipe(batch, batch_size=batch_size, truncation=True)
            for out in outputs:
                label = str(out.get("label", "")).upper()
                score = float(out.get("score", 0.0))
                polarity = score if "POS" in label else -score
                scores.append(polarity)
        return np.array(scores, dtype="float32")

    message_sentiments = _sentiment_polarity_batch(unique_texts, batch_size=64)
    
    np.save(message_sent_path, message_sentiments)
    np.save(message_ids_sent_path, unique_message_ids)
    print(f"Computed and saved sentiment for {len(message_sentiments)} messages")

# Create lookup dictionary
message_id_to_sentiment = dict(zip(unique_message_ids, message_sentiments))

# Look up sentiment for each pair
user_sent = np.array([message_id_to_sentiment[mid] for mid in df_pairs['user_message_id']])
assistant_sent = np.array([message_id_to_sentiment[mid] for mid in df_pairs['assistant_message_id']])

# Sentiment similarity mapped to [0, 1]
df_pairs["sentiment_similarity"] = 1.0 - (np.abs(user_sent - assistant_sent) / 2.0)
print(f"Computed sentiment similarity for {len(df_pairs)} pairs")

Device set to use cpu


Computing sentiment for 82940 unique messages (vs 149008 with old approach)
Efficiency gain: 1.80x fewer sentiment analyses
This will take a while on CPU. Consider running on GPU for faster processing.


Processing sentiment batches:   0%|          | 0/1296 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Linguistic Alignment (align2)


In [ ]:
# Linguistic alignment using align2 (align2-linguistic-alignment)
try:
    from align import Align
    aligner = Align()
except Exception:
    aligner = None

def _get_metric(result, name):
    if isinstance(result, dict):
        return result.get(name)
    return getattr(result, name, np.nan)

def compute_align_metrics(user_text, assistant_text):
    if aligner is None:
        return (np.nan, np.nan, np.nan)

    try:
        if hasattr(aligner, 'align'):
            result = aligner.align(user_text, assistant_text)
        elif hasattr(aligner, 'compute_alignment'):
            result = aligner.compute_alignment(user_text, assistant_text)
        else:
            return (np.nan, np.nan, np.nan)

        lex = _get_metric(result, 'lexical_alignment')
        syn = _get_metric(result, 'syntactic_alignment')
        sem = _get_metric(result, 'semantic_alignment')
        return (lex, syn, sem)
    except Exception:
        return (np.nan, np.nan, np.nan)

if not df_pairs.empty:
    metrics = [
        compute_align_metrics(u, a)
        for u, a in zip(df_pairs['user_text'], df_pairs['assistant_text'])
    ]
    df_pairs['lexical_alignment'] = [m[0] for m in metrics]
    df_pairs['syntactic_alignment'] = [m[1] for m in metrics]
    df_pairs['align2_semantic_alignment'] = [m[2] for m in metrics]
else:
    df_pairs['lexical_alignment'] = pd.Series(dtype=float)
    df_pairs['syntactic_alignment'] = pd.Series(dtype=float)
    df_pairs['align2_semantic_alignment'] = pd.Series(dtype=float)


## Plots

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.histplot(
    data=df_pairs,
    x="semantic_similarity",
    hue="direction",
    bins=50,
    stat="density",
    common_norm=False
)
plt.title("Semantic similarity by interaction direction")
plt.show()

In [ ]:
conv_means = (
    df_pairs
    .groupby(["share_id", "direction"])["semantic_similarity"]
    .mean()
    .unstack()
    .dropna()
)

conv_means["lead"] = (
    conv_means["user_to_assistant"]
    - conv_means["assistant_to_user"]
)


In [ ]:
sns.histplot(conv_means["lead"], bins=40)
plt.axvline(0, color="red", linestyle="--")
plt.title("Semantic lead (user→assistant minus assistant→user)")
plt.show()


In [ ]:
sns.lineplot(
    data=df_pairs,
    x="turn_index",
    y="semantic_similarity",
    hue="direction",
    errorbar="se"
)
plt.title("Semantic similarity over turns")
plt.show()


In [ ]:
sns.scatterplot(
    data=df_pairs,
    x="struct_user_text_length",
    y="semantic_similarity",
    alpha=0.2
)
plt.title("User message length vs semantic similarity")
plt.show()

In [ ]:
sns.boxplot(
    data=df_pairs,
    x="struct_has_any_block",
    y="semantic_similarity"
)
plt.title("Semantic similarity with vs without structured blocks")
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.boxplot(
    data=df_pairs,
    x="assistant_model_slug",
    y="semantic_similarity",
    hue="direction"
)
plt.xticks(rotation=45, ha="right")
plt.title("Semantic similarity by model and direction")
plt.tight_layout()
plt.show()

In [ ]:
top_subs = (
    df_pairs["subreddits"]
    .value_counts()
    .head(6)
    .index
)

df_sub = df_pairs[df_pairs["subreddits"].isin(top_subs)]


In [ ]:
g = sns.catplot(
    data=df_sub,
    x="assistant_model_slug",
    y="semantic_similarity",
    hue="direction",
    col="subreddits",
    kind="box",
    col_wrap=3,
    sharey=True
)

g.set_xticklabels(rotation=45)
g.fig.suptitle("Semantic similarity by model across subreddits", y=1.02)
plt.show()


In [ ]:
sns.boxplot(
    data=df_sub,
    x="subreddits",
    y="semantic_similarity",
    hue="direction"
)
plt.xticks(rotation=45, ha="right")
plt.title("Semantic similarity by subreddit and direction")
plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.boxplot(
    data=df_pairs,
    x="topic",
    y="semantic_similarity"
)
plt.title("Semantic similarity across KeyNMF topics")
plt.show()


In [ ]:
topic_summary = (
    df_pairs
    .groupby("topic")
    [["semantic_similarity", "sentiment_similarity", "lexical_alignment", "syntactic_alignment", "align2_semantic_alignment"]]
    .mean()
    .sort_values("semantic_similarity")
)

topic_summary.head(10)


## Aggregation and Outputs


In [ ]:
# Aggregate to conversation level (by direction)
if not df_pairs.empty:
    grouped = (
        df_pairs.groupby(['share_id', 'direction'])
        .agg(
            mean_semantic_similarity=('semantic_similarity', 'mean'),
            mean_sentiment_similarity=('sentiment_similarity', 'mean'),
            mean_lexical_alignment=('lexical_alignment', 'mean'),
            mean_syntactic_alignment=('syntactic_alignment', 'mean'),
            mean_align2_semantic_alignment=('align2_semantic_alignment', 'mean'),
            n_pairs=('turn_index', 'count'),
            conversation_length=('turn_index', 'max')
        )
        .reset_index()
    )

    def pivot_metric(metric_name, prefix):
        pivoted = grouped.pivot(index='share_id', columns='direction', values=metric_name)
        pivoted = pivoted.rename(columns={
            'user_to_assistant': f'{prefix}_user_to_assistant',
            'assistant_to_user': f'{prefix}_assistant_to_user'
        })
        return pivoted

    df_conversation = pd.concat([
        pivot_metric('mean_semantic_similarity', 'mean_semantic_similarity'),
        pivot_metric('mean_sentiment_similarity', 'mean_sentiment_similarity'),
        pivot_metric('mean_lexical_alignment', 'mean_lexical_alignment'),
        pivot_metric('mean_syntactic_alignment', 'mean_syntactic_alignment'),
        pivot_metric('mean_align2_semantic_alignment', 'mean_align2_semantic_alignment'),
        pivot_metric('n_pairs', 'n_pairs'),
        pivot_metric('conversation_length', 'conversation_length')
    ], axis=1).reset_index()

    df_conversation['semantic_lead'] = (
        df_conversation.get('mean_semantic_similarity_user_to_assistant')
        - df_conversation.get('mean_semantic_similarity_assistant_to_user')
    )

    metadata_cols = [
        'conversation_title',
        'conversation_create_time',
        'conversation_create_time_iso',
        'conversation_language',
        'default_model_slug',
        'subreddits',
        'assistant_model_slug'
    ] + [c for c in df_pairs.columns if c.startswith('struct_')]
    metadata_df = df_pairs[['share_id'] + metadata_cols].drop_duplicates('share_id')
    df_conversation = df_conversation.merge(metadata_df, on='share_id', how='left')
else:
    df_conversation = pd.DataFrame(
        columns=[
            'share_id',
            'mean_semantic_similarity_user_to_assistant',
            'mean_semantic_similarity_assistant_to_user',
            'mean_sentiment_similarity_user_to_assistant',
            'mean_sentiment_similarity_assistant_to_user',
            'mean_lexical_alignment_user_to_assistant',
            'mean_lexical_alignment_assistant_to_user',
            'mean_syntactic_alignment_user_to_assistant',
            'mean_syntactic_alignment_assistant_to_user',
            'mean_align2_semantic_alignment_user_to_assistant',
            'mean_align2_semantic_alignment_assistant_to_user',
            'n_pairs_user_to_assistant',
            'n_pairs_assistant_to_user',
            'conversation_length_user_to_assistant',
            'conversation_length_assistant_to_user',
            'semantic_lead'
        ]
    )

# Save outputs
turn_level_path = output_dir / 'alignment_turn_level.csv'
conversation_level_path = output_dir / 'alignment_conversation_level.csv'

df_pairs.to_csv(turn_level_path, index=False)
df_conversation.to_csv(conversation_level_path, index=False)
